In [1]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.8 MB 2.4 MB/s eta 0:00:01
   --------- ------------------------------ 0.5/1.8 MB 4.7 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.8 MB 6.0 MB/s eta 0:00:01
   --------------------------- ------------ 1.2/1.8 MB 6.5 MB/s eta 0:00:01
   -------------------------------- ------- 1.5/1.8 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 6.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.1
    Uninstalling pip-24.1.1:
      Successfully uninstalled pip-24.1.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install transformers psycopg2 numpy boto3 torch scikit-learn matplotlib nltk sentence-transformers pandas langchain lark pgvector psycopg2-binary tiktoken langchain_community huggingface_hub replicate keras tf-keras

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.7 MB 4.1 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.7 MB 5.6 MB/s eta 0:00:01
   --------------------- ------------------ 0.9/1.7 MB 6.6 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.7 MB 7.3 MB/s eta 0:00:01
   -------------------------------------- - 1.7/1.7 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.2/385.2 MB 14.1 MB/s eta 0:00:28
   ---------------------------------------- 0.7/385.2 MB 9.2 MB/s eta 0:00:42
   ---------------------------------------- 1.1/385.2 MB 8.8 MB/s eta 0:00:44
   ---------------------------------------- 1.5/385.2 MB 8.8 MB/s eta 0:00:44
   ----------

  You can safely remove it manually.


In [7]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
import psycopg2
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import json
import os
import pandas as pd

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [9]:
models = {
    "bart": {
        "model_name": "facebook/bart-large",
        "tokenizer": AutoTokenizer.from_pretrained("facebook/bart-large", trust_remote_code=True),
        "model": AutoModel.from_pretrained("facebook/bart-large", trust_remote_code=True)
    },
    "gte": {
        "model_name": "Alibaba-NLP/gte-large-en-v1.5",
        "tokenizer": AutoTokenizer.from_pretrained("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True),
        "model": AutoModel.from_pretrained("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)
    },
    "MiniLM": {
        "model_name": 'all-MiniLM-L12-v2',
        "model": SentenceTransformer('all-MiniLM-L12-v2')
    },
    "roberta": {
        "model_name": 'sentence-transformers/nli-roberta-large',
        "model": SentenceTransformer('sentence-transformers/nli-roberta-large')
    },
    "e5-large":{
        "model_name": 'intfloat/e5-large',
        "tokenizer": AutoTokenizer.from_pretrained('intfloat/e5-large', trust_remote_code=True),
        "model": AutoModel.from_pretrained('intfloat/e5-large', trust_remote_code=True)
    }
}

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [12]:
current_directory = os.getcwd()
with open(os.path.join(current_directory, "movies.json"), "r", encoding="utf-8") as f:
    movies = json.load(f)

movies_data = []
for movie in movies["films"]["film"]:

    roles = movie.get("role", [])
    if isinstance(roles, dict):  # If 'roles' is a dictionary, make it a single-item list
        roles = [roles]

    # Extract actor information
    actors = []
    for role in roles:
        actor_info = role.get("acteur", {})
        if "__text" in actor_info:
            actors.append(actor_info["__text"])

    movies_data.append({
        "title": movie.get("titre", ""),
        "year": movie.get("annee", ""),
        "country": movie.get("pays", ""),
        "language": movie.get("langue", ""),
        "duration": movie.get("duree", ""),
        "summary": movie.get("synopsis", ""),
        "genre": movie.get("genre", ""),
        "director": movie.get("realisateur", {"__text": ""}).get("__text", ""),
        "writers": movie.get("scenariste", []),
        "actors": actors,
        "poster": movie.get("affiche", ""),
        "id": movie.get("id", "")
    })

In [13]:
def preprocess(text):
    # Example preprocessing step simplified for demonstration
    tokens = text.split()
    # Assuming stopwords are already loaded to avoid loading them in each process
    stopwords_set = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stopwords_set]
    return ' '.join(tokens)

In [14]:
def normalize_embeddings(embeddings):
    """ Normalize the embeddings to unit vectors. """
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    normalized_embeddings = embeddings / norms
    return normalized_embeddings

In [15]:
def generate_embedding(movies_data, model_key, normalize=True):
    model_config = models[model_key]
    if 'tokenizer' in model_config:
        # Handle HuggingFace transformer models
        movie_texts = [
            f"{preprocess(movie['title'])} {movie['year']} {' '.join(movie['genre'])} "
            f"{' '.join(movie['actors'])} {movie['director']} "
            f"{preprocess(movie['summary'])} {movie['country']}"
            for movie in movies_data
        ]
        inputs = model_config['tokenizer'](movie_texts, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model_config['model'](**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    else:
        # Handle Sentence Transformers
        movie_texts = [
            f"{preprocess(movie['title'])} {movie['year']} {' '.join(movie['genre'])} "
            f"{' '.join(movie['actors'])} {movie['director']} "
            f"{preprocess(movie['summary'])} {movie['country']}"
            for movie in movies_data
        ]
        embeddings = model_config['model'].encode(movie_texts)

    if normalize:
        embeddings = normalize_embeddings(embeddings)

    return embeddings


In [16]:
embeddings_bart = generate_embedding(movies_data, 'bart')
embeddings_bart = np.array(embeddings_bart)
print("BART embeddings shape:", embeddings_bart.shape)
print("BART embeddings:", embeddings_bart[0])

BART embeddings shape: (631, 1024)
BART embeddings: [ 0.02980587  0.01581689 -0.0162874  ... -0.00557002 -0.02872263
 -0.02430014]


In [17]:
embeddings_gte = generate_embedding(movies_data, 'gte')
embeddings_gte = np.array(embeddings_gte)
print("GTE embeddings shape:", embeddings_gte.shape)
print("GTE embeddings:", embeddings_gte[0])

GTE embeddings shape: (631, 1024)
GTE embeddings: [ 0.00271679  0.00554605  0.01195132 ...  0.04074343 -0.00983375
 -0.02142335]


In [18]:
embeddings_MiniLM = generate_embedding(movies_data, 'MiniLM')
embeddings_MiniLM = np.array(embeddings_MiniLM)
print("MiniLM embeddings shape:", embeddings_MiniLM.shape)
print("MiniLM embeddings:", embeddings_MiniLM[0])

MiniLM embeddings shape: (631, 384)
MiniLM embeddings: [-7.04024499e-03  6.07011141e-03  1.28376424e-01 -4.50421683e-02
  4.66892868e-02  1.10597022e-01  5.81973493e-02  2.08956115e-02
  5.28918505e-02  1.14241503e-01 -9.48251039e-02 -4.47546691e-02
 -1.81262288e-02  1.00744896e-01 -4.28932756e-02  5.07416651e-02
 -1.08549166e-02  7.32663646e-02 -2.75690667e-02  8.67665000e-03
 -7.07044676e-02  4.54404689e-02 -3.44804972e-02 -8.39157868e-03
  1.86072644e-02 -7.11285621e-02  2.02321932e-02 -5.20065464e-02
  3.22543341e-03 -1.99356731e-02  5.68822511e-02 -2.74389284e-03
 -2.67803054e-02  2.91455481e-02 -4.49042680e-04  5.57996966e-02
 -1.30207360e-01  1.55704385e-02 -2.01300066e-02 -5.58847636e-02
  1.33942459e-02 -3.11912205e-02  1.25691712e-01 -9.65598375e-02
 -1.80415250e-02 -1.05671175e-02 -5.14752939e-02 -3.55250835e-02
  4.20107394e-02  8.37841257e-02 -7.53990039e-02 -2.90898960e-02
 -1.50384801e-03  8.55621509e-03 -1.42899957e-02 -8.19770396e-02
 -5.21209054e-02  1.44929150e-02  1

In [19]:
embeddings_roberta = generate_embedding(movies_data, 'roberta')
embeddings_roberta = np.array(embeddings_roberta)
print("RoBERTa embeddings shape:", embeddings_roberta.shape)
print("RoBERTa embeddings:", embeddings_roberta[0])

RoBERTa embeddings shape: (631, 1024)
RoBERTa embeddings: [ 0.02298054  0.04175862 -0.01646351 ...  0.01556577 -0.0256803
  0.06172808]


In [20]:
embeddings_e5_large = generate_embedding(movies_data, 'e5-large')
embeddings_e5_large = np.array(embeddings_e5_large)
print("e5-large embeddings shape:", embeddings_e5_large.shape)
print("e5-large embeddings:", embeddings_e5_large[0])

e5-large embeddings shape: (631, 1024)
e5-large embeddings: [-0.01276984 -0.05366214  0.04731817 ...  0.01435485  0.02704632
  0.03876564]


# Create connection to the database

In [22]:
conn = psycopg2.connect(database="postgres", host="localhost", user="postgres", password="postgres", port="5432")
cur = conn.cursor()

In [23]:
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()
cur.execute("CREATE EXTENSION IF NOT EXISTS cube;")
conn.commit()

In [24]:
def setup_database():
    cur.execute('DROP TABLE IF EXISTS movies')
    cur.execute('''
        CREATE TABLE movies (
            id SERIAL PRIMARY KEY,
            title TEXT NOT NULL,
            actors TEXT,
            year INTEGER,
            country TEXT,
            language TEXT,
            duration INTEGER,
            summary TEXT,
            genre TEXT[],
            director TEXT,
            scenarists TEXT[],
            poster TEXT,
            embedding_bart VECTOR(1024),
            embedding_gte VECTOR(1024),
            embedding_MiniLM VECTOR(384),
            embedding_roberta VECTOR(1024),
            embedding_e5_large VECTOR(1024)
        );
    ''')
    conn.commit()

setup_database()


# Insert

In [25]:
def insert_movies(movie_data, embeddings_bart, embeddings_gte, embeddings_MiniLM, embeddings_roberta, embeddings_e5_large):
    for movie, emb_bart, emb_gte, emb_MiniLM , emb_roberta, emb_e5_large in zip(movie_data, embeddings_bart, embeddings_gte, embeddings_MiniLM, embeddings_roberta, embeddings_e5_large):
        # Joining actors into a single string separated by commas
        actor_names = ', '.join(movie['actors'])
        # Convert list of genres into a PostgreSQL array format
        genre_array = '{' + ', '.join([f'"{g}"' for g in movie['genre']]) + '}'
        # Convert list of scenarists into a PostgreSQL array format
        scenarist_array = '{' + ', '.join([f'"{s}"' for s in movie['writers']]) + '}'
        # Convert embeddings to a string properly formatted as a list
        embedding_bart_str = '[' + ', '.join(map(str, emb_bart)) + ']'
        embedding_gte_str = '[' + ', '.join(map(str, emb_gte)) + ']'
        embedding_MiniLM_str = '[' + ', '.join(map(str, emb_MiniLM)) + ']'
        embedding_roberta_str = '[' + ', '.join(map(str, emb_roberta)) + ']'
        embedding_e5_large_str = '[' + ', '.join(map(str, emb_e5_large)) + ']'

        cur.execute('''
            INSERT INTO movies (title, actors, year, country, language, duration, summary, genre, director, scenarists, poster, embedding_bart, embedding_gte, embedding_MiniLM, embedding_roberta, embedding_e5_large)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', (
            movie['title'], actor_names, movie['year'], movie['country'], movie['language'],
            movie['duration'], movie['summary'], genre_array, movie['director'],
            scenarist_array, movie['poster'], embedding_bart_str, embedding_gte_str, embedding_MiniLM_str, embedding_roberta_str, embedding_e5_large_str
        ))
    conn.commit()

In [26]:
insert_movies(movies_data, embeddings_bart, embeddings_gte, embeddings_MiniLM, embeddings_roberta, embeddings_e5_large)